<a href="https://colab.research.google.com/github/bhadreshpsavani/ExploringOCR/blob/master/ExploringLayoutLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 2.0MB 4.4MB/s 
     |████████████████████████████████| 3.2MB 19.8MB/s 
     |████████████████████████████████| 890kB 36.4MB/s 


In [2]:
from transformers import LayoutLMTokenizer, LayoutLMForTokenClassification
import torch
tokenizer = LayoutLMTokenizer.from_pretrained('microsoft/layoutlm-base-uncased')
model = LayoutLMForTokenClassification.from_pretrained('microsoft/layoutlm-base-uncased')

Some weights of the model checkpoint at microsoft/layoutlm-base-uncased were not used when initializing LayoutLMForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing LayoutLMForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LayoutLMForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft

In [3]:
words = ["Hello", "world"]
normalized_word_boxes = [637, 773, 693, 782], [698, 773, 733, 782]
token_boxes = []
for word, box in zip(words, normalized_word_boxes):
    word_tokens = tokenizer.tokenize(word)
    token_boxes.extend([box] * len(word_tokens))

In [7]:
token_boxes

[[0, 0, 0, 0],
 [637, 773, 693, 782],
 [698, 773, 733, 782],
 [1000, 1000, 1000, 1000]]

In [8]:
word_tokens

['world']

In [4]:
# add bounding boxes of cls + sep tokens
token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]

In [5]:
encoding = tokenizer(' '.join(words), return_tensors="pt")
input_ids = encoding["input_ids"]
attention_mask = encoding["attention_mask"]
token_type_ids = encoding["token_type_ids"]
bbox = torch.tensor([token_boxes])
token_labels = torch.tensor([1,1,0,0]).unsqueeze(0) # batch size of 1

In [6]:
outputs = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids,
                labels=token_labels)
loss = outputs.loss
logits = outputs.logits

In [9]:
logits

tensor([[[-0.0344,  0.2428],
         [ 0.1178,  0.2391],
         [ 0.1211,  0.1931],
         [-0.0339,  0.2423]]], grad_fn=<AddBackward0>)

In [10]:
loss

tensor(0.6923, grad_fn=<NllLossBackward>)

In [11]:
from transformers import LayoutLMTokenizer, LayoutLMForSequenceClassification
import torch
tokenizer = LayoutLMTokenizer.from_pretrained('microsoft/layoutlm-base-uncased')
model = LayoutLMForSequenceClassification.from_pretrained('microsoft/layoutlm-base-uncased')
words = ["Hello", "world"]
normalized_word_boxes = [637, 773, 693, 782], [698, 773, 733, 782]
token_boxes = []
for word, box in zip(words, normalized_word_boxes):
    word_tokens = tokenizer.tokenize(word)
    token_boxes.extend([box] * len(word_tokens))
# add bounding boxes of cls + sep tokens
token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]
encoding = tokenizer(' '.join(words), return_tensors="pt")
input_ids = encoding["input_ids"]
attention_mask = encoding["attention_mask"]
token_type_ids = encoding["token_type_ids"]
bbox = torch.tensor([token_boxes])
sequence_label = torch.tensor([1])
outputs = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids,
                labels=sequence_label)
loss = outputs.loss
logits = outputs.logits
logits

Some weights of the model checkpoint at microsoft/layoutlm-base-uncased were not used when initializing LayoutLMForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing LayoutLMForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LayoutLMForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LayoutLMForSequenceClassification were not initialized from the model checkpoint 

tensor([[ 0.1121, -0.0101]], grad_fn=<AddmmBackward>)

In [14]:
from transformers import LayoutLMTokenizer, LayoutLMForMaskedLM
import torch
tokenizer = LayoutLMTokenizer.from_pretrained('microsoft/layoutlm-base-uncased')
model = LayoutLMForMaskedLM.from_pretrained('microsoft/layoutlm-base-uncased')
words = ["Hello", "[MASK]"]
normalized_word_boxes = [637, 773, 693, 782], [698, 773, 733, 782]
token_boxes = []
for word, box in zip(words, normalized_word_boxes):
    word_tokens = tokenizer.tokenize(word)
    token_boxes.extend([box] * len(word_tokens))
# add bounding boxes of cls + sep tokens
token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]
encoding = tokenizer(' '.join(words), return_tensors="pt")
input_ids = encoding["input_ids"]
attention_mask = encoding["attention_mask"]
token_type_ids = encoding["token_type_ids"]
bbox = torch.tensor([token_boxes])
labels = tokenizer("Hello world", return_tensors="pt")["input_ids"]
outputs = model(input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids,
                labels=labels)
loss = outputs.loss

In [15]:
outputs

MaskedLMOutput([('loss', tensor(11.0031, grad_fn=<NllLossBackward>)),
                ('logits',
                 tensor([[[ 0.5534,  0.1442,  0.0901,  ..., -0.0590,  0.8132,  0.7086],
                          [-1.6607, -0.8606, -1.0854,  ...,  0.2116,  0.2717, -0.9807],
                          [-1.4802, -0.6127, -1.2147,  ...,  0.5059,  0.1404, -1.1180],
                          [ 0.5582,  0.1473,  0.0929,  ..., -0.0526,  0.8160,  0.7153]]],
                        grad_fn=<AddBackward0>))])